In [42]:
# import pandas
import pandas as pd

# Pull data from AADT excel spreadsheet and put into dataframes

In [43]:
# GLOBAL VARIABLES

# excel spreadsheet copied to data/udot folder from https://drive.google.com/file/d/1rDXm0ObugGR1zXgWUuVbzWHNt-Xs1xru/view
fnExcelAADTHistory = 'data/AADTHistory.xlsx'

In [44]:
# create connection to Excel file and get list of sheets
xl_file1 = pd.ExcelFile(fnExcelAADTHistory)

# display sheet names
print('Excel sheet names: ' + str(xl_file1.sheet_names))

Excel sheet names: ['AADT_2021', 'AADT_Rounded', 'AADT_Unrounded']


Note: AADT for 2021 is in a separate sheet called 'AADT_2021'. AADT for years 1981-2020 are in AADT_Rounded and AADT_Unrounded. AADT unrounded is what we will use, since we want to forecast from unrounded values

In [45]:
# put data in to series of dataframes (dfsExcelSheets)
dfsExcelSheets = {sheet_name: xl_file1.parse(sheet_name) for sheet_name in xl_file1.sheet_names}

c:\Users\bhereth\Anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


# Get AADT for years prior to 2021

In [46]:
# use temp dataframe _df as a copy of excel sheet
_df = dfsExcelSheets['AADT_Unrounded'].copy()

# get all columns that start with 'AADT'
aadt_cols = [col for col in _df.columns if col.startswith('AADT')]

# define the id columns that we want to keep
id_vars = ['Route', 'Beg MP', 'End MP']

# melt the DataFrame - melt is a reverse pivot
dfUnder2021 = _df.melt(id_vars=id_vars, value_vars=aadt_cols, var_name='YEAR', value_name='AADT')

display (dfUnder2021)

,Route,Beg MP,End MP,YEAR,AADT
0,0006PM,0.000,46.038,AADT2020,430
1,0006PM,46.038,77.556,AADT2020,385
2,0006PM,77.556,82.897,AADT2020,552
3,0006PM,82.897,83.911,AADT2020,2061
4,0006PM,83.911,87.694,AADT2020,3409
...,...,...,...,...,...
181475,3468PM,0.000,5.125,AADT1981,0
181476,3469PM,0.000,6.930,AADT1981,0
181477,3470PM,0.000,1.039,AADT1981,0
181478,3478PM,0.000,2.040,AADT1981,0


In [47]:
# replace 'AADT' in the 'YEAR' column and convert to integer
dfUnder2021['YEAR'] = dfUnder2021['YEAR'].str.replace('AADT', '').astype(int)

# rename columns to match 2021 data
dfUnder2021.rename(columns={'Route':'ROUTE_ID','Beg MP':'FROM_MEASURE','End MP':'TO_MEASURE'}, inplace=True)

display(dfUnder2021)

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT
0,0006PM,0.000,46.038,2020,430
1,0006PM,46.038,77.556,2020,385
2,0006PM,77.556,82.897,2020,552
3,0006PM,82.897,83.911,2020,2061
4,0006PM,83.911,87.694,2020,3409
...,...,...,...,...,...
181475,3468PM,0.000,5.125,1981,0
181476,3469PM,0.000,6.930,1981,0
181477,3470PM,0.000,1.039,1981,0
181478,3478PM,0.000,2.040,1981,0


# Process 2021 AADTs

AADTs for 2021 are different from other years and are in their own sheet called 'AADT 2021'

In [48]:
# show snippet of 2021 sheet
dfsExcelSheets['AADT_2021']

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT_2021
0,0006PM,0.000,46.038,474
1,0006PM,46.038,77.556,424
2,0006PM,77.556,82.897,609
3,0006PM,82.897,83.911,2272
4,0006PM,83.911,87.694,3852
...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886
4569,0015PC29501,0.000,0.166,1000
4570,0015PC30554,0.000,1.135,35025
4571,0092NC00101,0.000,3.294,12201


In [49]:
# PREPARE 2021 DATA
df2021 = dfsExcelSheets['AADT_2021'].copy(0)

# add column for YEAR
df2021['YEAR'] = 2021

# rename AADT_2021 to AADT to match data from other years
df2021.rename(columns={'AADT_2021':'AADT'}, inplace=True)
df2021

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,AADT,YEAR
0,0006PM,0.000,46.038,474,2021
1,0006PM,46.038,77.556,424,2021
2,0006PM,77.556,82.897,609,2021
3,0006PM,82.897,83.911,2272,2021
4,0006PM,83.911,87.694,3852,2021
...,...,...,...,...,...
4568,0015PC29402,0.000,0.199,8886,2021
4569,0015PC29501,0.000,0.166,1000,2021
4570,0015PC30554,0.000,1.135,35025,2021
4571,0092NC00101,0.000,3.294,12201,2021


# Combine under 2021 and 2021

In [50]:
# combine 2021 and under 2021
dfAADT = pd.concat([dfUnder2021,df2021])

#dfAADT.reset_index(inplace=True)
dfAADT.fillna('0',inplace=True)

# filter out 0 AADTs
dfAADT = dfAADT[dfAADT['AADT']>0]

# round to nearest hundredth
dfAADT['FROM_MEASURE'] = dfAADT['FROM_MEASURE'].round(2)
dfAADT['TO_MEASURE'  ] = dfAADT['TO_MEASURE'  ].round(2)

display(dfAADT)

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT
0,0006PM,0.00,46.04,2020,430
1,0006PM,46.04,77.56,2020,385
2,0006PM,77.56,82.90,2020,552
3,0006PM,82.90,83.91,2020,2061
4,0006PM,83.91,87.69,2020,3409
...,...,...,...,...,...
4568,0015PC29402,0.00,0.20,2021,8886
4569,0015PC29501,0.00,0.17,2021,1000
4570,0015PC30554,0.00,1.14,2021,35025
4571,0092NC00101,0.00,3.29,2021,12201


In [51]:
# Filter rows with 'ROUTE_ID' containing 'PM': Positive Direction & Mainline
dfAADT_filtered = dfAADT[dfAADT['ROUTE_ID'].str.contains('PM')].copy()

# Trim 'ROUTE_ID' down to the first four characters in order to match with SEGID
dfAADT_filtered['ROUTE_ID'] = dfAADT_filtered['ROUTE_ID'].str[:4]

# sort by fields SEGID and YEAR
dfAADT_filtered = dfAADT_filtered.sort_values(by=['ROUTE_ID','FROM_MEASURE','TO_MEASURE','YEAR'])

dfAADT_filtered

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT
176943,0006,0.0,46.04,1981,325
172406,0006,0.0,46.04,1982,335
167869,0006,0.0,46.04,1983,430
163332,0006,0.0,46.04,1984,580
158795,0006,0.0,46.04,1985,585
...,...,...,...,...,...
18147,3483,0.0,1.70,2017,990
13610,3483,0.0,1.70,2018,1030
9073,3483,0.0,1.70,2019,1042
4536,3483,0.0,1.70,2020,1041


In [52]:
# CHECK FOR DUPLICATES

# Define the columns you want to check for duplicates
columns_to_check = ['ROUTE_ID', 'FROM_MEASURE', 'TO_MEASURE', 'YEAR']

# Use the duplicated method to find duplicates in those columns
duplicates = dfAADT_filtered.duplicated(subset=columns_to_check, keep=False)

# Filter the DataFrame to only include the duplicates
dfAADT_duplicates = dfAADT_filtered[duplicates]

display(dfAADT_duplicates)

,ROUTE_ID,FROM_MEASURE,TO_MEASURE,YEAR,AADT


# Export

In [53]:
dfAADT_filtered.to_csv('intermediate/historic-aadt.csv', index=False)